# Amazon SageMaker scikit-learn Bring Your Own Model
_**Hosting a pre-trained scikit-learn Model in Amazon SageMaker scikit-learn Container**_

---

---

## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-trained scikit-learn model with the Amazon SageMaker scikit-learn container to quickly create a hosted endpoint for that model.
We use the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html. The California Housing dataset was originally published in:

> Pace, R. Kelley, and Ronald Barry. "Sparse spatial auto-regressions." Statistics & Probability Letters 33.3 (1997): 291-297.

---
## Setup

Ensure we have the latest verion of the SageMaker Python SDK.

In [1]:
!pip install -U sagemaker

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 475 kB 26.6 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.72.3-py2.py3-none-any.whl size=654043 sha256=87c03e2c1f8789c90a06faa1b2bd07c8bd42ac1ab677a664e9218a658de55094
  Stored in directory: /root/.cache/pip/wheels/b0/af/a0/5c66d761bdb3fdaf0e10e9ab0c260d60f72098eeb3d778deac
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.72.2
    Uninstalling sagemaker-2.72.2:
      Successfully uninstalled sagemaker-2.72.2


Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data.
* The S3 bucket that you want to use for training and model data.

In [2]:
import os
import time
import boto3
import re
import json
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role, image_uris, ModelPackage
from sagemaker.sklearn.model import SKLearnModel, SKLearnPredictor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-sklearn-byo-model"

print(f"bucket: {bucket}")
print(f"sagemaker version: {sagemaker.__version__}")

bucket: sagemaker-us-east-1-862774760132
sagemaker version: 2.72.3


## Prepare data for model inference

We load the California housing dataset from sklearn, and will use it to invoke SageMaker Endpoint

In [3]:
data = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

# we don't train a model, so we will need only the testing data
testX = pd.DataFrame(X_test, columns=data.feature_names)

testX.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44
3,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72
4,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93
5,4.7147,12.0,5.251483,0.975089,2400.0,2.846975,34.08,-117.61
6,5.0839,36.0,6.221719,1.095023,670.0,3.031674,33.89,-118.02
7,3.6908,38.0,4.962825,1.048327,1011.0,3.758364,33.92,-118.08
8,4.8036,4.0,3.924658,1.035959,1050.0,1.797945,37.39,-122.08
9,8.1132,45.0,6.879056,1.011799,943.0,2.781711,34.18,-118.23


## Create the pre-trained model file

In [4]:
import sys
!{sys.executable} -m pip install lightgbm

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [5]:
import lightgbm as lgb
import joblib

model = lgb.LGBMRegressor()

model.fit(X_train, y_train)

print(model.predict(X_test[:5, :]))
print(y_test[:5])

model_file_name = "model.joblib"

joblib.dump(model, model_file_name)

[0.63346062 0.96964309 4.99226567 2.50300127 2.43031094]
[0.477   0.458   5.00001 2.186   2.78   ]


['model.joblib']

## Write the Inference Script

When using endpoints with the Amazon SageMaker managed `Scikit Learn` container, we need to provide an entry point script for inference that will **at least** load the saved model.

After the SageMaker model server has loaded your model by calling `model_fn`, SageMaker will serve your model. Model serving is the process of responding to inference requests, received by SageMaker `InvokeEndpoint` API calls.


We will implement also the `predict_fn()` function that takes the deserialized request object and performs inference against the loaded model.

We will now create this script and call it `inference.py` and store it at the root of a directory called `code`.

**Note:** You would modify the script below to implement your own inferencing logic.

Additional information on model loading and model serving for scikit-learn on SageMaker can be found in the [SageMaker Scikit-learn Model Server documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#deploy-a-scikit-learn-model)

There are also several functions for hosting which we won't define,
 - `input_fn()` - Takes request data and deserializes the data into an object for prediction.
 - `output_fn()` - Takes the result of prediction and serializes this according to the response content type.

These will take on their default values as described [SageMaker Scikit-learn Serve a Model documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#serve-a-model)

In [6]:
model_code_path = "./code"
model_code_inference = "inference.py"

In [7]:
!mkdir -p $model_code_path

In [8]:
%%writefile $model_code_path/$model_code_inference

import os
import joblib


def predict_fn(input_object, model):
    ###########################################
    # Do your custom preprocessing logic here #
    ###########################################

    print("calling model")
    predictions = model.predict(input_object)
    return predictions


def model_fn(model_dir):
    print("loading model.joblib from: {}".format(model_dir))
    loaded_model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return loaded_model

Writing ./code/inference.py


### Installing additional Python dependencies

It also may be necessary to supply a `requirements.txt` file to ensure any necessary dependencies are installed in the container along with the script. For this script, in addition to the Python standard libraries, we showcase how to install the `boto3` and `requests` libraries.

In [9]:
%%writefile $model_code_path/requirements.txt

lightgbm

Writing ./code/requirements.txt


## Package the pre-trained model in `model.tar.gz` and upload it to S3
The model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`, and needs to be tar-zipped.

In [10]:
model_tar_name = "model.tar.gz"
!tar czvf $model_tar_name $model_file_name 

model.joblib


In [11]:
key = os.path.join(prefix, model_tar_name)
s3 = boto3.client("s3")
s3.upload_file(model_tar_name, bucket, key)
model_data = f"s3://{bucket}/{key}"
print(f"model data: {model_data}")

model data: s3://sagemaker-us-east-1-862774760132/sagemaker/DEMO-sklearn-byo-model/model.tar.gz


### Create the model

Here we showcase the process of creating a model from s3 artifacts, that could be used to deploy a model that was trained in a different session or even out of SageMaker.

In [12]:
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="0.23-1",
    py_version="py3",
    source_dir=model_code_path,
    entry_point=model_code_inference,
    sagemaker_session=sagemaker.Session(),  # Required for model.register().
)

### Register the model version
Create a model group.

In [13]:
sm_client = boto3.client("sagemaker", region_name=region)
model_package_group_name = "scikit-housing-prediction" + str(round(time.time()))
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "For predicting ln(median house value)",
}

create_model_pacakge_group_response = sm_client.create_model_package_group(
    **model_package_group_input_dict
)
model_package_group_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
print("ModelPackageGroup Arn : {}".format(model_package_group_arn))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:862774760132:model-package-group/scikit-housing-prediction1641997896


Register the model to the model group.

In [14]:
model_package = model.register(
    content_types=["text/csv", "application/json"],
    response_types=["text/csv", "application/json"],
    inference_instances=["ml.t2.medium"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    description="Predict house values",
    approval_status="Approved",
)
model_package_arn = model_package.model_package_arn
print(model_package_arn)

arn:aws:sagemaker:us-east-1:862774760132:model-package/scikit-housing-prediction1641997896/1


### Batch transform data.

Upload the test data to S3.

In [15]:
np.savetxt("X_test.csv", X_test, delimiter=",")

bucket = sagemaker.Session().default_bucket()
X_test_prefix = f"{prefix}/input/X_test.csv"
s3.upload_file("X_test.csv", bucket, X_test_prefix)
X_test_S3 = f"s3://{bucket}/{X_test_prefix}"
output_path = f"s3://{bucket}/{prefix}/output"

print(X_test_S3)
print(output_path)

s3://sagemaker-us-east-1-862774760132/sagemaker/DEMO-sklearn-byo-model/input/X_test.csv
s3://sagemaker-us-east-1-862774760132/sagemaker/DEMO-sklearn-byo-model/output


Create a transformer and process the data.

In [16]:
model_from_package = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker.Session(),
)

transformer = model_from_package.transformer(
    instance_count=1, instance_type="ml.m5.large", output_path=output_path
)

In [17]:
%%time

transformer.transform(X_test_S3, content_type="text/csv", split_type="Line")
transformer.wait()

.............................2022-01-12 14:36:20,962 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2022-01-12 14:36:20,965 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2022-01-12 14:36:20,966 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      ret

Download the batch transform results and print them.

In [18]:
s3.download_file(bucket, f"{prefix}/output/X_test.csv.out", "X_test.csv.out")

with open("X_test.csv.out", "r") as f:
    X_test_out = json.load(f)

print(X_test_out[:5])
print(y_test[:5])

[0.6334606231537258, 0.9696430900210699, 4.992265674900781, 2.5030012731023397, 2.4303109395315596]
[0.477   0.458   5.00001 2.186   2.78   ]


### Create a real-time endpoint
Create an endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 5-10 minutes to complete.

In [19]:
%%time

endpoint_name = f"scikit-housing-prediction-{str(round(time.time()))}"
model_from_package.deploy(
    instance_type="ml.t2.medium", initial_instance_count=1, endpoint_name=endpoint_name
)
predictor = SKLearnPredictor(endpoint_name=endpoint_name)

-----------!CPU times: user 158 ms, sys: 13 ms, total: 170 ms
Wall time: 5min 32s


Let's generate the prediction for the test data generated earlier.

In [20]:
# the SKLearnPredictor does the serialization from pandas for us
predictions = predictor.predict(testX[data.feature_names])
print(predictions[:5])
print(y_test[:5])

[0.63346062 0.96964309 4.99226567 2.50300127 2.43031094]
[0.477   0.458   5.00001 2.186   2.78   ]


If you're ready to be done with this endpoint, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [21]:
predictor.delete_endpoint()